In [7]:
# Import libraries
import pandas as pd
import numpy as np
import gensim
import multiprocessing
from sklearn import utils
from ast import literal_eval

# Gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

# Progress bar
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

In [8]:
# Import dataset
df = pd.read_csv('../Data/4. Labeling/labeling.csv')

# Mengonversi kolom tokenisasi menjadi list
df['text_tokenize'] = df['text_tokenize'].apply(literal_eval)

text_tokenize = df['text_tokenize']

# Menampilkan total dataset
print("Total Dataset :", len(df))

df.head()

Total Dataset : 4776


,video_id,comment_text,text_cleaned,text_removal,text_normalized,text_manual_replaced,text_tokenize,label
0,video1,Baiknya bagaimana menurut kalian?,baiknya bagaimana menurut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,baik bagaimana turut kalian,"[baik, bagaimana, turut, kalian]",0
1,video1,Betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,betul dok betul dok,"[betul, dok, betul, dok]",0
2,video1,Betul dok mungkin efek sa di,betul dok mungkin efek sa di,betul dok mungkin efek sa,betul dok mungkin efek sa,betul dok mungkin efek sa,"[betul, dok, mungkin, efek, sa]",0
3,video1,"Bpom aneh, tidak kerja. Kenapa ko seperti itu.",bpom aneh tidak kerja kenapa ko seperti itu,bpom aneh kerja ko itu,bpom aneh kerja kok itu,bpom aneh kerja kok itu,"[bpom, aneh, kerja, kok, itu]",-1
4,video1,Benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obatny...,benar banget anak sy aj hrs sirup minum obat d...,benar banget anak saya saja harus sirup minum ...,benar banget anak aku saja harus sirup minum o...,"[benar, banget, anak, aku, saja, harus, sirup,...",0


Word2Vec

Word2Vec dengan arsitektur Continuous Bag of Words (CBOW)

In [11]:
# Jumlah core CPU yang tersedia: memeriksa jumlah core CPU untuk paralelisasi
cores = multiprocessing.cpu_count()

# Membuat model Word2Vec dengan arsitektur CBOW: mengatur parameter untuk model Word2Vec
model_ug_cbow = Word2Vec(
    sg=0,  # Menggunakan CBOW (sg=0)
    vector_size=100,  # Ukuran vektor fitur
    negative=5,  # Jumlah negative sampling
    window=2,  # Jarak maksimum antara kata target dan kata-kata yang mengelilinginya
    min_count=1,  # Minimum jumlah kemunculan kata untuk dimasukkan ke dalam model
    workers=cores,  # Jumlah worker yang akan digunakan
    alpha=0.065,  # Learning rate awal
    min_alpha=0.065,  # Learning rate minimum
    hs=1,  # Menggunakan hierarchical softmax
)

# Membangun kosakata dari semua komentar yang telah dikonversi menjadi menjadi list token
model_ug_cbow.build_vocab(text_tokenize)

"""
bagian kode ini berfungsi untuk mempersiapkan dan mengatur model Word2Vec dengan arsitektur Continuous Bag of Words (CBOW). 
Parameter model diatur sesuai kebutuhan. Kemudian, kosakata dibangun dari semua komentar yang telah dikonversi menjadi list token.

100%|██████████| 4776/4776 [00:00<00:00, 1595412.23it/s]


In [12]:
%%time
# Melatih model Word2Vec dengan arsitektur CBOW sebanyak 30 epoch 
for epoch in range(30):
    # Mengacak urutan data untuk setiap epoch: membuat variasi dalam data latih untuk setiap epoch
    shuffled_data = utils.shuffle(text_tokenize)
    
    # Melatih model dengan data yang telah diacak: proses pembelajaran model
    model_ug_cbow.train(shuffled_data, total_examples=len(text_tokenize), epochs=1)
    
    # Mengurangi learning rate sebesar 0.002 setiap epoch: untuk mengoptimalkan proses pembelajaran
    model_ug_cbow.alpha -= 0.002
    
    # Menyesuaikan learning rate minimum agar sama dengan learning rate saat ini
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

"""
bagian kode ini berfungsi untuk melatih model Word2Vec dengan arsitektur CBOW. 
Pelatihan dilakukan sebanyak 30 epoch, dengan setiap epoch melibatkan pengacakan data dan penurunan learning rate. 
Ini membantu model untuk belajar dari variasi dalam data dan mengoptimalkan proses pembelajaran 
dengan mengurangi learning rate seiring berjalannya waktu. 
Penurunan learning rate ini membantu model untuk konvergen lebih cepat ke solusi optimal.

100%|██████████| 4776/4776 [00:00<00:00, 2376556.64it/s]


CPU times: total: 8.17 s
Wall time: 2.31 s


In [13]:
# Membuat model Word2Vec dengan arsitektur Skip-gram
model_ug_sg = Word2Vec(
    sg=1,  # Menggunakan Skip-gram (sg=1)
    vector_size=100,  # Ukuran vektor fitur
    negative=5,  # Jumlah negative sampling
    window=2,  # Jarak maksimum antara kata target dan kata-kata yang mengelilinginya
    min_count=1,  # Minimum jumlah kemunculan kata untuk dimasukkan ke dalam model
    workers=cores,  # Jumlah worker yang akan digunakan
    alpha=0.065,  # Learning rate awal
    min_alpha=0.065,  # Learning rate minimum
    hs=1,  # Menggunakan hierarchical softmax
)

# Membangun kosakata dari semua komentar yang telah dikonversi menjadi  list token
model_ug_sg.build_vocab(text_tokenize)

"""
bagian kode ini berfungsi untuk mempersiapkan dan mengatur model Word2Vec dengan arsitektur Skip-Gram. 
Parameter model diatur sesuai kebutuhan. Kemudian, kosakata dibangun dari semua komentar yang telah dikonversi menjadi list token.


100%|██████████| 4776/4776 [00:00<00:00, 4779765.19it/s]


In [14]:
%%time
# Melatih model Word2Vec dengan arsitektur Skip-gram sebanyak 30 epoch
for epoch in range(30):
    # Mengacak urutan data untuk setiap epoch
    shuffled_data = utils.shuffle(text_tokenize)
    
    # Melatih model dengan data yang telah diacak
    model_ug_sg.train(shuffled_data, total_examples=len(text_tokenize), epochs=1)
    
    # Mengurangi learning rate sebesar 0.002 setiap epoch
    model_ug_sg.alpha -= 0.002
    
    # Menyesuaikan learning rate minimum agar sama dengan learning rate saat ini
    model_ug_sg.min_alpha = model_ug_sg.alpha
    
"""
Pelatihan dilakukan sebanyak 30 epoch, dengan setiap epoch melibatkan pengacakan data dan penurunan learning rate. 
Ini membantu model untuk belajar dari variasi dalam data dan mengoptimalkan proses pembelajaran 
dengan mengurangi learning rate seiring berjalannya waktu. 
Penurunan learning rate ini membantu model untuk konvergen lebih cepat ke solusi optimal.


100%|██████████| 4776/4776 [00:00<00:00, 2394739.50it/s]


CPU times: total: 19.1 s
Wall time: 3.76 s


In [15]:
# Menyimpan model Word2Vec CBOW yang telah dilatih
model_ug_cbow.save('../Model/w2v_model_ug_cbow.word2vec')

# Menyimpan model Word2Vec Skip-gram yang telah dilatih
model_ug_sg.save('../Model/w2v_model_ug_sg.word2vec')

CBOW: memprediksi kata target (focus word) berdasarkan kata-kata konteks yang mengelilinginya. Sebagai contoh, diberikan kalimat "Kucing itu sangat lucu", dan kata target adalah "sangat", maka CBOW akan menggunakan "Kucing", "itu", "lucu" sebagai konteks untuk memprediksi kata "sangat".

Skip-gram: mencoba memprediksi kata-kata konteks (kata-kata sekitar) berdasarkan kata target (kata pusat). Sebagai contoh, diberikan kalimat "Kucing itu sangat lucu", dan kata target adalah "sangat", maka Skip-gram akan menggunakan "sangat" untuk memprediksi kata "Kucing", "itu", dan "lucu".

Hemat waktu: Menyimpan model menghindari kebutuhan untuk melatihnya lagi, yang bisa memakan waktu lama.

Reproduktibilitas: Model yang disimpan bisa digunakan kembali untuk tujuan yang sama atau berbeda, dan bisa dibagi dengan orang lain, mendukung penelitian dan pengembangan.

Pengujian: Model yang disimpan memungkinkan untuk pengujian dan validasi lebih lanjut, memungkinkan perbandingan dan optimalisasi performa.